# Inference

In this notebook we demonstrate how to start the simulation from arbitrary SMPL(-X) pose using linear blend-skinning (LBS) initialization. If you don't want to use LBS (for example, you are using non-SMPL(-X) body meshes),  please see `Inference_from_mesh_sequence.ipynb`.

## Choose a garment

First, you need to choose a garment to simulate.

Its template and some auxiliary data should be stored in the .pkl file under `DEFAULTS.data_root/aux_data/garment_dicts/`

You can choose from the list of outfits already provided in this folder:

![ccraft_garments](static/ccraft_garments.png)

Or you can import a new garment from an `.obj` file

We also provide `.obj` files for all garments usen in the paper in `DEFAULTS.data_root/aux_data/garment_meshes/` directory.
Note that these `.obj` files only have demonstrational purpose. 
For inference and training we use garment data stored in the .pkl files under `DEFAULTS.data_root/aux_data/garments_dicts/`, not the .obj files

## Or add your own garment from an `.obj` file

[GarmentImport.ipynb](GarmentImport.ipynb) notebook demostrates how you can import garments from .obj files.

To do this, you'll either need 
* a garment geometry aligned with the canonical SMPL(-X) body OR
* a garment geometry aligned with an arbitrary SMPL(-X) body and the corresponging SMPL(-X) parameters.

[GarmentImport.ipynb](GarmentImport.ipynb) discusses both of these cases.

In the end you will get a .pkl file containing all information required to simulate your garment. We call such .pkl files "garment dictionaries" or "garment dicts".


If you want to create an outfit from several garments whose geometries may intersect, use [Untanglement.ipynb](Untanglement.ipynb) to order and untangle their geometries and combine them into a single outfit. (You'' anyway first need to create garment dicts for each garment as described in GarmentImport.ipynb).

# Generate rollout for one sequence

Once we have created a garment dict file for our garment (or you can use one of the garments that are already under `DEFAULTS.data_root/aux_data/garment_dicts/`), we can generate a rollout sequence for them using a trained HOOD/ContourCraft model.

We provide 3 pretrained models and corresponding configuration files for each of them. The weights of the trained models are located in `DEFAULTS.data_root/trained_models`. The configuration files are in  `DEFAULTS.project_dir/configs`

| model file      | config name           | comments                                                                                                                                                                                                                            |
|-----------------|-----------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| cvpr_submission | hood_cvpr                  | HOOD model used in the CVPR paper. No multi-layer simulation. Use it if you want to compare to the HOOD paper.                                                                                                                                                           |
| postcvpr        | hood_final              | Newer HOOD model trained using refactored code with minor bug fixes. No multi-layer simulation. Use it if you want to use HOOD model in a downstream task.|
| **contourcraft**        | **contourcraft**              | **Model from the ContourCraft paper. Can simulate multi-layer outfits**. |

## Choose pose sequence 

This repository supports inference over .npz sequences from CMU split of AMASS dataset.

You can download them [here](https://amass.is.tue.mpg.de/). Use gendered SMPL+H sequences if you want to use SMPL model and gendered SMPL-X sequences if you want to use SMPL-X one.


### create validation config and create `Runner` object

In [ ]:
from utils.validation import apply_material_params
from utils.validation import load_runner_from_checkpoint
from utils.arguments import load_params
from utils.common import move2device
from utils.io import pickle_dump
from utils.defaults import DEFAULTS
from pathlib import Path


# Set material paramenters, see configs/cvpr.yaml for the training ranges for each parameter
material_dict = dict()
material_dict['density'] = 0.20022
material_dict['lame_mu'] = 23600.0
material_dict['lame_lambda'] = 44400
material_dict['bending_coeff'] = 3.962e-05


# ====================================================================================================

models_dir = Path(DEFAULTS.data_root) / 'trained_models'

# Choose the model and the configuration file

# config_name = 'hood_cvpr'
# checkpoint_path = models_dir / 'hood_cvpr.pth'

# config_name = 'hood_final'
# checkpoint_path = models_dir / 'hood_final.pth'

config_name = 'contourcraft'
checkpoint_path = models_dir / 'contourcraft.pth'


# ====================================================================================================


# load the config from .yaml file and load .py modules specified there
modules, experiment_config = load_params(config_name)

# modify the config to use it in validation 
experiment_config = apply_material_params(experiment_config, material_dict)

# load Runner object and the .py module it is declared in
runner_module, runner = load_runner_from_checkpoint(checkpoint_path, modules, experiment_config)


/local/home/agrigorev/Workdir/contourcraft_private/utils/validation.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(checkpoint_path)


### create one-sequence dataloader

Here you'll need to choose a garment by setting `garment_name` variable. The garment name should correspond to a `.pkl` file under `DEFAULTS.data_root/aux_data/garments_dicts/`

Note that it can also be a comma-separated list of garments, then they'll be combined into a single outfit. For example:
```
garment_name = 'smplx/cindy_020::bottom_skirt, smplx/cindy_020::top_blouse'
```

**To test for SMPL poses, use**

```
sequence_path =  'path/to/SMPLH/AMASS/CMU/01/01_01_poses.npz'
garment_name = 'longskirt'
sequence_loader = 'cmu_npz_smpl'
``` 

In [ ]:
# file with the pose sequence
from utils.validation import create_postcvpr_one_sequence_dataloader

# If True, the SMPL(-X) poses are slightly modified to avoid hand-body self-penetrations. The technique is adopted from the code of SNUG 
separate_arms = True

# path to the pose sequence file
CMU_path = 'path/to/AMASS/CMU'
sequence_path =  Path(CMU_path) / '01/01_01_stageii.npz'

# name of the garment to simulate 
# should correspond to a pkl file under DEFAULTS.data_root/aux_data/garments_dicts/
garment_name = 'smplx/cindy_020_combined_test'

# It can be a comma-separated list of individual garments
# garment_name = 'smplx/cindy_020::bottom_skirt, smplx/cindy_020::top_blouse'


# gender of the body model, sould be the same as the one used to create the garment
gender = 'female'

# ====================================================================================================

# Choose the type of the pose sequence you want to use: 'cmu_npz_smpl', 'cmu_npz_smplx'

# to use AMASS SMPL-X pose sequence
sequence_loader = 'cmu_npz_smplx'

# to use AMASS SMPL pose sequence
# sequence_loader = 'cmu_npz_smpl



# ====================================================================================================


dataloader = create_postcvpr_one_sequence_dataloader(sequence_path, garment_name, sequence_loader=sequence_loader, 
                                            obstacle_dict_file=None, gender=gender)

In [6]:
sequence = next(iter(dataloader))
sequence = move2device(sequence, 'cuda:0')
trajectories_dict = runner.valid_rollout(sequence,  bare=True, n_steps=200)

100%|██████████| 200/200 [00:38<00:00,  5.22it/s]


In [ ]:
# Save the sequence to disk
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output.pkl'
print(f"Rollout saved into {out_path}")
pickle_dump(dict(trajectories_dict), out_path)

Rollout saved into /data/agrigorev/02_Projects/ccraft_data/temp/output_hood_old.pkl


### write a video

Finally, we can render a video of the generated sequence with [aitviewer](https://github.com/eth-ait/aitviewer)

Or you can render it interactively using `python utils/show.py rollout_path=PATH_TO_SEQUENCE`

In [1]:
from utils.show import write_video 
from aitviewer.headless import HeadlessRenderer

# Careful!: creating more that one renderer in a single session causes an error
renderer = HeadlessRenderer()

/data/agrigorev/miniforge3/envs/ccraft/lib/python3.10/site-packages/trimesh/geometry.py:7: UserWarning: A NumPy version >=1.23.5 and <2.5.0 is required for this version of SciPy (detected version 1.23.1)
  import scipy.sparse


In [2]:
out_path = Path(DEFAULTS.data_root) / 'temp' / 'output.pkl'
out_video = Path(DEFAULTS.data_root) / 'temp' / 'output.mp4'
write_video(out_path, out_video, renderer)

NameError: name 'Path' is not defined